In [1]:
import pandas  as pd
import numpy as np

In [7]:
biographies=pd.read_csv('/Users/macbook/Desktop/BA6/Database/db2018imdb/biographies.csv', sep=',', error_bad_lines=False)
people=pd.read_csv('/Users/macbook/Desktop/BA6/Database/newfiles/people.csv', sep=',')

b'Skipping line 5664: expected 15 fields, saw 21\nSkipping line 8638: expected 15 fields, saw 22\n'
b'Skipping line 80094: expected 15 fields, saw 22\nSkipping line 124625: expected 15 fields, saw 22\n'
b'Skipping line 151837: expected 15 fields, saw 21\nSkipping line 156757: expected 15 fields, saw 17\nSkipping line 172733: expected 15 fields, saw 22\nSkipping line 193522: expected 15 fields, saw 22\n'
b'Skipping line 198145: expected 15 fields, saw 22\nSkipping line 215281: expected 15 fields, saw 21\nSkipping line 220659: expected 15 fields, saw 17\nSkipping line 253950: expected 15 fields, saw 22\n'
b'Skipping line 265088: expected 15 fields, saw 17\nSkipping line 307082: expected 15 fields, saw 22\n'
b'Skipping line 336443: expected 15 fields, saw 22\n'
b'Skipping line 421643: expected 15 fields, saw 17\n'


In [8]:
cols_to_check = ['Spouse','Nickname','BiographicalBooks']
biographies[cols_to_check] = biographies[cols_to_check].replace({'\[':'','\]':''},regex=True)

biographies = biographies.replace('', np.nan, regex=True)
biographies.drop_duplicates(keep='first', inplace=True)

biographies["BioId"] = biographies.index + 1
biographies.rename(columns={'Name': 'FullName'}, inplace=True)
result = pd.merge(biographies, people, how='left', on='FullName')

In [9]:
result = result[np.isnan(result['pid'])]
result['pid']=(result.FullName).rank(method='dense')+people['pid'].max()
result[['pid']] = result[['pid']].astype(int)
result= result[['pid','FullName']]
people = pd.concat([people, result])

In [10]:
people.to_csv('people.csv', sep=',', encoding='utf-8', index=False)

In [11]:
biographies = pd.merge(biographies, people, how='left', on='FullName')
bio = biographies.drop(['Spouse','BiographicalBooks'],axis=1)
bio.to_csv('biographies.csv', sep=',', encoding='utf-8', index=False)

In [7]:
married = biographies[['Spouse','pid']].copy()
married.dropna(subset=['Spouse'], inplace=True)
married[['Spouse']] = married[['Spouse']].astype(str)

def splitDataFrameList(df,target_column,separator):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = []
        i=0
        for c in target_column:
            split_row.append(row[c].split(separator))
            i+=1
            
        for s in range (0,len(split_row[0])):
            j=0
            new_row = row.to_dict()
            for c in target_column:
                new_row[c] = split_row[j][s]
                j+=1
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    global new_df
    new_df = pd.DataFrame(new_rows)
    return new_df

married = splitDataFrameList( married, ['Spouse'] , '|')
married = married.replace('', np.nan, regex=True)
married = married.replace('nan', np.nan, regex=True)
married.dropna(subset=['Spouse'], inplace=True)
married['sid']=(married.Spouse).rank(method='dense')
married[['sid']] = married[['sid']].astype(int)
married.rename(columns={'Spouse': 'name'}, inplace=True)

married.to_csv('married.csv', sep=',', encoding='utf-8', index=False)

/Applications/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Applications/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2450: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [8]:
BiographicalBooks = biographies[['BiographicalBooks','pid']].copy()

BiographicalBooks[['BiographicalBooks']] = BiographicalBooks[['BiographicalBooks']].astype(str)

BiographicalBooks = splitDataFrameList( BiographicalBooks, ['BiographicalBooks'] , '|')
BiographicalBooks = BiographicalBooks.replace('', np.nan, regex=True)
BiographicalBooks = BiographicalBooks.replace('nan', np.nan, regex=True)
BiographicalBooks.dropna(subset=['BiographicalBooks'], inplace=True)
BiographicalBooks['bookid']=(BiographicalBooks.BiographicalBooks).rank(method='dense')
BiographicalBooks[['bookid']] = BiographicalBooks[['bookid']].astype(int)
BiographicalBooks.rename(columns={'BiographicalBooks': 'name'}, inplace=True)

BiographicalBooks.to_csv('BiographicalBooks.csv', sep=',', encoding='utf-8', index=False)

/Applications/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2450: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [9]:
BiographicalBooks

,name,pid,bookid
17,Lord Lovat. _March Past._ 1978.,103,15694
47,Jan Omdahl. _The Swing of Things - the book ab...,278,13327
64,Jan E.Hansen. _Kjære Per Aabel._ Oslo J.W.Capp...,361,13318
65,Per Aabel. _Den stundesløse Per Aabel._ Oslo G...,361,18003
66,Per Aabel. _Du verden._ Oslo Prent Forlag 1950.,361,18004
72,Florence Aadland. _The Beautiful Pervert._ 1965.,387,6548
73,Florence Aadland. _The Big Love._,389,6549
95,Goran Schildt ed.. _Alvar Aalto In His Own Wor...,503,7319
121,Willie Aames and Maylo Upton-Ames with Carolyn...,613,21735
155,Aaron Chester. _Garlic is Life A Memoir with R...,2347694,2020


In [18]:
married = biographies[['Spouse','pid']].copy()
married.dropna(subset=['Spouse'], inplace=True)
married[['Spouse']] = married[['Spouse']].astype(str)
pd.set_option('display.max_colwidth', -1)
